### Toy Lindblad identification as POP solved with Moment SDP

In [1]:
using LinearAlgebra, JuMP, MosekTools
using Optim

Consider SDP optimization problem of the form:
\begin{align} 
&\operatorname{minimize}{ \left( \mathbf{trace}{(C X)} \right) }\\
&\operatorname{subject to} { y_i := \left( \mathbf{trace}{(A_i X)} \right) = b_i}, \quad i = 1... m, \quad
X \succcurlyeq 0
\end{align}

where $X\in\mathbf{S}^{n}$ is the decision variable, and each of the $A_{i}$ matrices and $C$ are also in $\mathbf{S}^{n}$  By the notation $\mathbf{S}^{n}$, we denote the set of all symmetric $n \times n$ matrices.

It could be solved with the following function:

In [2]:
function solve_SDP(A, b, C; quiet = false, term_tolerance = 1e-7)
    
    start_time = time()
    
    n = size(A)[1] # = size(C)[1]
    m = length(b)

    model = Model(optimizer_with_attributes(Mosek.Optimizer, "QUIET" => quiet, "INTPNT_CO_TOL_DFEAS" => term_tolerance))

    #set_silent(model)

    @variable(model, X[1:n, 1:n], PSD)

    @objective(model, Min, tr(C * X));
    
    @constraint(model, y[j=1:m], tr(A[:, (j - 1) * n + 1:j * n]* X) == b[j])

    optimize!(model)

    status = JuMP.termination_status(model)
    X_sol = JuMP.value.(X)
    obj_value = JuMP.objective_value(model)
    dual_sol = dual.(y)
    run_time = time() - start_time

    return model, status, run_time, X_sol, dual_sol, obj_value
    
end

solve_SDP (generic function with 1 method)

In [3]:
using DynamicPolynomials
@polyvar x[1:2]

(PolyVar{true}[x₁, x₂],)

$\min_{x \in \mathbb{R}^2 } p(x)$

In [4]:
function p(x)
    x[1]^4 + x[1]^2 + 2x[1]x[2] + 2x[1]^2 * x[2] + 2x[2]^2
end

p (generic function with 1 method)

In [5]:
obj = p(x)

x₁⁴ + 2x₁²x₂ + x₁² + 2x₁x₂ + 2x₂²

Numerical solution with BFGS:

In [6]:
result = optimize(obj, zeros(2), BFGS())
@show x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ = Optim.minimizer(result)
p([x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ])

(x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ) = Optim.minimizer(result) = [0.0, 0.0]


0.0

#### Formulating SDP

Having polynomial optimization problem:

$\min_{x \in \mathbb{R}^2 } p(x)$

we can write the first and second linear matrix inequality (LMI) relaxation for it:
    
$\min_{y \in \mathbb{R}^6 }$
    
$\operatorname{s.t.}{y_{00} = 1, M_1(y)\succcurlyeq 0}$

Which is equivalent to:
\begin{align} 
&\operatorname{minimize}{ \left( \mathbf{trace}{(C X)} \right) }\\
&\operatorname{subject to} { y_i := \left( \mathbf{trace}{(A_i X)} \right) = b_i}, \quad i = 1... m, \quad
X \succcurlyeq 0
\end{align}

where

$X = M_2(y) =\begin{pmatrix} 
    y_{00} & y_{10} & y_{01} & y_{20} & y_{11} & y_{02}\\
    y_{10} & y_{20} & y_{11} & y_{30} & y_{21} & y_{12}\\
    y_{01} & y_{11} & y_{02} & y_{21} & y_{12} & y_{03}\\
    y_{20} & y_{30} & y_{21} & y_{40} & y_{31} & y_{22}\\
    y_{11} & y_{21} & y_{12} & y_{31} & y_{22} & y_{13}\\
    y_{02} & y_{12} & y_{03} & y_{22} & y_{13} & y_{04}\\
    \end{pmatrix} \succcurlyeq 0 $

In [7]:
@polyvar y[0:4,0:4]

X     = [ y[1,1] y[2,1] y[1,2] y[3,1] y[2,2] y[1,3]
          y[2,1] y[3,1] y[2,2] y[4,1] y[3,2] y[2,3]
          y[1,2] y[2,2] y[1,3] y[3,2] y[2,3] y[1,4]
          y[3,1] y[4,1] y[3,2] y[5,1] y[4,2] y[3,3]
          y[2,2] y[3,2] y[2,3] y[4,2] y[3,3] y[2,4]
          y[1,3] y[2,3] y[1,4] y[3,3] y[2,4] y[1,5] ]

6×6 Matrix{PolyVar{true}}:
 y₀₋₀  y₁₋₀  y₀₋₁  y₂₋₀  y₁₋₁  y₀₋₂
 y₁₋₀  y₂₋₀  y₁₋₁  y₃₋₀  y₂₋₁  y₁₋₂
 y₀₋₁  y₁₋₁  y₀₋₂  y₂₋₁  y₁₋₂  y₀₋₃
 y₂₋₀  y₃₋₀  y₂₋₁  y₄₋₀  y₃₋₁  y₂₋₂
 y₁₋₁  y₂₋₁  y₁₋₂  y₃₋₁  y₂₋₂  y₁₋₃
 y₀₋₂  y₁₋₂  y₀₋₃  y₂₋₂  y₁₋₃  y₀₋₄

In [8]:
obj

x₁⁴ + 2x₁²x₂ + x₁² + 2x₁x₂ + 2x₂²

In [9]:
C = [0   0   0    1/3  1/2  2/3 
     0   1/3 1/2  0    1/2  0 
     0   1/2 2/3  1/2  0    0         
     1/3 0   1/2  1    0    0 
     1/2 1/2 0    0    0    0 
     2/3 0   0    0    0    0 ]
@show rank(C)
tr(C*X)

rank(C) = 6


y₂₋₀ + y₄₋₀ + 2.0y₁₋₁ + 2.0y₂₋₁ + 2.0y₀₋₂

In [10]:
eigen(C)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
6-element Vector{Float64}:
 -0.9062084980711499
 -0.3834657745555047
  0.13237675713716002
  0.8035835880657819
  0.8127470836468974
  1.540966843776813
vectors:
6×6 Matrix{Float64}:
 -0.675832   0.231201   0.107987  -0.594515  -0.243844   -0.255397
 -0.209996  -0.604923  -0.229154   0.331548  -0.574661   -0.311929
  0.031842   0.379922   0.549757   0.475443  -0.0474584  -0.56931
  0.109829  -0.193014  -0.358306  -0.201356   0.560795   -0.683567
  0.488755   0.487294  -0.457658  -0.163621  -0.503542   -0.184081
  0.497187  -0.40195    0.543839  -0.49322   -0.200016   -0.110492

and as $y_{00}=1$:

$A_1 =\begin{pmatrix} 
1 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
\end{pmatrix}, \quad b_1 = 1$

In [11]:
model, status, run_time, X_sol, dual_sol, obj_value = solve_SDP(A, b, C)

LoadError: UndefVarError: A not defined

In [12]:
obj_value

LoadError: UndefVarError: obj_value not defined

In [13]:
X_sol

LoadError: UndefVarError: X_sol not defined

In [14]:
eigen(X_sol)

LoadError: UndefVarError: X_sol not defined

In [15]:
coefficients(1+x[1]+x[2]+x[1]^2+x[1]*x[2]+x[2]^2)

6-element Vector{Int64}:
 1
 1
 1
 1
 1
 1

In [16]:
rmons = reverse(monomials(1+x[1]+x[2]+x[1]^2+x[1]*x[2]+x[2]^2))

6-element Vector{Monomial{true}}:
 1
 x₂
 x₁
 x₂²
 x₁x₂
 x₁²

In [17]:
rmons*rmons'

6×6 Matrix{Monomial{true}}:
 1     x₂     x₁     x₂²     x₁x₂    x₁²
 x₂    x₂²    x₁x₂   x₂³     x₁x₂²   x₁²x₂
 x₁    x₁x₂   x₁²    x₁x₂²   x₁²x₂   x₁³
 x₂²   x₂³    x₁x₂²  x₂⁴     x₁x₂³   x₁²x₂²
 x₁x₂  x₁x₂²  x₁²x₂  x₁x₂³   x₁²x₂²  x₁³x₂
 x₁²   x₁²x₂  x₁³    x₁²x₂²  x₁³x₂   x₁⁴

In [18]:
monomials(obj)[3:3]*monomials(obj)[1:3]'

1×3 Matrix{Monomial{true}}:
 x₁⁶  x₁⁴x₂  x₁⁴

In [19]:
coefficients(obj)*coefficients(obj)'

5×5 Matrix{Int64}:
 1  2  1  2  2
 2  4  2  4  4
 1  2  1  2  2
 2  4  2  4  4
 2  4  2  4  4

In [20]:
C = [7         -1       -1       0           0      -1 
     -1          0        0       0           0      0 
     -1          0        0       0           0      0         
     0           0        0       0           0      0 
     0           0        0       1           0      0 
     -1          0        0       0           0      1 ]

tr(C*X)

7y₀₋₀ - 2y₁₋₀ - 2y₀₋₁ + y₃₋₁ - 2y₀₋₂ + y₀₋₄

In [21]:
obj

x₁⁴ + 2x₁²x₂ + x₁² + 2x₁x₂ + 2x₂²

This is SDP problem, 



and as $y_{00}=1$:

$A_1 =\begin{pmatrix} 
1 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
\end{pmatrix}, \quad b_1 = 1$


In [22]:
rank(C₂)

LoadError: UndefVarError: C₂ not defined

In [23]:
@polyvar y[1:6,1:6]

(PolyVar{true}[y₁₋₁ y₁₋₂ … y₁₋₅ y₁₋₆; y₂₋₁ y₂₋₂ … y₂₋₅ y₂₋₆; … ; y₅₋₁ y₅₋₂ … y₅₋₅ y₅₋₆; y₆₋₁ y₆₋₂ … y₆₋₅ y₆₋₆],)

Additional constaint for $y_{00}$:

In [24]:
A = zeros(6,6)
A[1,1] = 1
b = 1
A

6×6 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

Now we can solve SDP

In [25]:
model, status, run_time, X_sol, dual_sol, obj_value = solve_SDP(A, b, C)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective se

(A JuMP Model
Minimization problem with:
Variables: 21
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`Vector{VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Mosek
Names registered in the model: X, y, MathOptInterface.DUAL_INFEASIBLE, 6.293999910354614, [9.648864805190962e-13 3.112805641442038e-8 … -1.1242504152157373e-34 -4.195261145106528e-7; 3.112805641442038e-8 0.6330493779142524 … -1.3076438217747849e-16 0.018883753382189096; … ; -1.1242504152157373e-34 -1.3076438217747849e-16 … 2.119258751751721 2.662633334820755e-29; -4.195261145106528e-7 0.018883753382189096 … 2.662633334820755e-29 0.3029348143741101], [-0.7502497056251967], -1.1924489262842486)

In [26]:
obj_value

-1.1924489262842486

In [27]:
status

DUAL_INFEASIBLE::TerminationStatusCode = 3

In [28]:
X_sol

6×6 Matrix{Float64}:
  9.64886e-13   3.11281e-8    3.11281e-8   …  -1.12425e-34  -4.19526e-7
  3.11281e-8    0.633049     -0.366951        -1.30764e-16   0.0188838
  3.11281e-8   -0.366951      0.633049         1.30764e-16   0.0188838
  8.72454e-35   2.87073e-17  -2.87073e-17     -1.49538      -3.5658e-29
 -1.12425e-34  -1.30764e-16   1.30764e-16      2.11926       2.66263e-29
 -4.19526e-7    0.0188838     0.0188838    …   2.66263e-29   0.302935

In [37]:
eigen(X_sol)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
6-element Vector{Float64}:
 3.5915818341696296e-13
 0.2520758220163274
 0.31695774818685835
 0.6238742965465974
 0.9999999999999979
 3.61464320695682
vectors:
6×6 Matrix{Float64}:
  1.0          -9.2834e-7    -1.1073e-6   …   1.64237e-16  -2.70458e-17
 -2.17179e-7   -0.626047      0.328732        0.707107     -2.22045e-16
 -2.17179e-7   -0.626047      0.328732       -0.707107     -1.66533e-16
 -1.11022e-16  -3.38237e-17   2.1939e-17     -3.39806e-16  -0.707107
 -3.76794e-17   0.0           0.0            -2.68484e-16   0.707107
  1.41195e-6    0.464898      0.885364    …  -2.14058e-14  -5.43951e-20

#### Compare methods

In [29]:
x₁ˢᵈᵖ = sqrt(X_sol[2,2]) # ω = √x₁²

0.7956440019972829

In [30]:
x₂ˢᵈᵖ = sqrt(X_sol[3,3]) # a = √x₂²

#x₁ˢᵈᵖ, x₂ˢᵈᵖ = X_sol[1,2], X_sol[1,3]

0.7956440019972597

In [31]:
x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ

(0.0, 0.0)

In [32]:
x₁ᵃⁿᵃˡⁱᵗ, x₂ᵃⁿᵃˡⁱᵗ

LoadError: UndefVarError: x₁ᵃⁿᵃˡⁱᵗ not defined

In [33]:
[x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ] - [x₁ᵃⁿᵃˡⁱᵗ, x₂ᵃⁿᵃˡⁱᵗ]

LoadError: UndefVarError: x₁ᵃⁿᵃˡⁱᵗ not defined

The precission of SDP is around $10^{-6}$:

In [34]:
[x₁ˢᵈᵖ, x₂ˢᵈᵖ] - [x₁ᵃⁿᵃˡⁱᵗ, x₂ᵃⁿᵃˡⁱᵗ]

LoadError: UndefVarError: x₁ᵃⁿᵃˡⁱᵗ not defined

In [35]:
result = optimize(obj, [x₁ˢᵈᵖ, x₂ˢᵈᵖ], BFGS())
@show x₁ˢᵈᵖᵇᶠᵍˢ, x₂ˢᵈᵖᵇᶠᵍˢ = Optim.minimizer(result)
p([x₁ˢᵈᵖᵇᶠᵍˢ, x₂ˢᵈᵖᵇᶠᵍˢ])

(x₁ˢᵈᵖᵇᶠᵍˢ, x₂ˢᵈᵖᵇᶠᵍˢ) = Optim.minimizer(result) = [0.9999999996696052, -0.9999999994500549]


0.0

In [36]:
using TSSOS
opt,sol,data = tssos_first(obj, variables(obj), QUIET=true, solution=true);
previous_sol = sol

************************TSSOS************************
TSSOS is launching...
optimum = -2.6376938269342776e-10

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Global optimality certified!


2-element Vector{Float64}:
 -4.757243104594542e-10
  3.0862650598908017e-10